# 使用代理引擎、ADK 和 Vertex Express 模式免費使用 Vertex 會話服務

本教學是[代理程式開發套件](https://google.github.io/adk-docs/get-started/)的[快速入門範例](https://google.github.io/adk-docs/get-started/quickstart/)的延伸。

我們將著手建構一個**天氣機器人代理程式**，建立一個可以查詢天氣的單一代理程式，我們將把它與一些免費的 Vertex 服務（如 VertexAiSessionService）連接起來，讓使用者可以在 Vertex 上儲存他們的會話！

**ADK 又是什麼？**

提醒您，ADK 是一個 Python 框架，旨在簡化由大型語言模型 (LLM) 驅動的應用程式的開發。它提供了強大的建構區塊，用於建立能夠推理、規劃、利用工具、與使用者動態互動以及在團隊中有效協作的代理程式。

**在本教學中，您將掌握：**

*   ✅ **工具定義與使用：** 製作 Python 函式 (`tools`)，賦予代理程式特定能力 (如擷取資料)，並指示代理程式如何有效地使用它們。
*   ✅ **代理程式引擎 API：** 部署本地代理程式並免費使用代理程式引擎的功能，如 VertexAiSession 服務。

**最終狀態期望：**

完成本教學後，您將建構一個可以使用代理程式引擎建構區塊的功能性天氣代理程式。

**先決條件：**

*   ✅ **對 Python 程式設計有扎實的理解。**
*   ✅ **熟悉大型語言模型 (LLM)、API 和代理程式的概念。**
*   ❗ **至關重要：完成 ADK 快速入門教學或具備同等的 ADK 基礎知識 (Agent、Runner、SessionService、基本工具使用)。** 本教學直接建立在這些概念之上。
*   ✅ **您打算使用的 LLM 的 API 金鑰** (例如，用於 Gemini 的 Google AI Studio、OpenAI Platform、Anthropic Console)。


---

**準備好建立您的代理程式團隊了嗎？讓我們開始吧！**

In [ ]:
# 設定與安裝
# 安裝 ADK

!pip install google-adk -q

print("安裝完成。")

In [ ]:
# @title 匯入必要的函式庫
import os
import asyncio
from google import adk
from google.adk.agents import Agent
from google.adk.sessions import VertexAiSessionService
from google.adk.memory import VertexAiMemoryBankService
from google.adk.runners import Runner
from google.genai import types # 用於建立訊息內容/部分

import warnings
# 忽略所有警告
warnings.filterwarnings("ignore")

# 設定 API 金鑰 (請替換為您的實際金鑰！)
我們可以透過 Vertex Express 模式免費使用像 VertexAiSessionService 這樣的 Vertex 服務並存取模型！在此處使用您的 Google 帳戶註冊：https://cloud.google.com/vertex-ai/generative-ai/docs/start/express-mode/overview，即可免費存取某些服務和模型，無需新增信用卡。

Vertex Express 模式與 ADK 相結合，可以免費建立進階代理程式！您將可以免費存取某些 Gemini 模型和代理程式引擎服務，如會話和記憶體，而無需帳單帳戶！

In [ ]:
# Gemini API 金鑰 (從 Vertex Express 模式取得)
easygcp_api_key = "INSERT_API_KEY" #@param {type:"string"}
os.environ["GOOGLE_API_KEY"] = easygcp_api_key
# 將 vertex 設定為 true
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"

# --- 驗證金鑰 (可選檢查) ---
print("API 金鑰已設定：")
print(f"Google API 金鑰已設定：{'是' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'INSERT API KEY HERE' else '否 (請替換預留位置！)'}")

在建立代理程式時，我們需要選擇我們想要使用的模型。Vertex Express 模式 API 金鑰允許免費使用多個 Gemini 模型，您可以使用此處列出的任何模型：https://cloud.google.com/vertex-ai/generative-ai/docs/start/express-mode/overview#models

In [ ]:
# --- 定義模型常數以便於使用 ---

# 為 EasyGCP 使用允許清單中的模型，我們將使用 gemini 2.0
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash-001"

print("\n環境已設定。")

---

## 步驟 1：您的第一個代理程式 - 基本天氣查詢

讓我們從建構天氣機器人的基本元件開始：一個能夠執行特定任務——查詢天氣資訊的單一代理程式。這涉及建立兩個核心部分：

1. **一個工具：** 一個 Python 函式，賦予代理程式擷取天氣資料的*能力*。  
2. **一個代理程式：** AI「大腦」，理解使用者的請求，知道它有一個天氣工具，並決定何時以及如何使用它。

---

**1\. 定義工具 (`get_weather`)**

在 ADK 中，**工具**是賦予代理程式超越純文字生成之具體能力的建構區塊。它們通常是執行特定動作的常規 Python 函式，例如呼叫 API、查詢資料庫或執行計算。

我們的第一個工具將提供一個*模擬*的天氣報告。這讓我們可以專注於代理程式的結構，而無需立即需要外部 API 金鑰。稍後，您可以輕鬆地將此模擬函式換成呼叫真實天氣服務的函式。

**核心概念：文件字串至關重要！** 代理程式的 LLM 在很大程度上依賴函式的**文件字串**來理解：

* 工具*做什麼*。  
* *何時*使用它。  
* 它需要*什麼參數* (`city: str`)。  
* 它傳回*什麼資訊*。

**最佳實踐：** 為您的工具撰寫清晰、具描述性且準確的文件字串。這對於 LLM 正確使用該工具至關重要。

In [ ]:
# @title 定義 get_weather 工具
def get_weather(city: str) -> dict:
    """擷取指定城市的目前天氣報告。

    Args:
        city (str): 城市名稱 (例如，「New York」、「London」、「Tokyo」)。

    Returns:
        dict: 一個包含天氣資訊的字典。
              包含一個 'status' 鍵 ('success' 或 'error')。
              如果 'success'，則包含一個 'report' 鍵，其中包含天氣詳細資訊。
              如果 'error'，則包含一個 'error_message' 鍵。
    """
    print(f"--- 工具：為城市呼叫 get_weather：{city} ---") # 記錄工具執行
    city_normalized = city.lower().replace(" ", "") # 基本正規化

    # 模擬天氣資料
    mock_weather_db = {
        "newyork": {"status": "success", "report": "紐約天氣晴朗，溫度為 25°C。"},
        "london": {"status": "success", "report": "倫敦多雲，溫度為 15°C。"},
        "tokyo": {"status": "success", "report": "東京有小雨，溫度為 18°C。"},
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"抱歉，我沒有 '{city}' 的天氣資訊。"}

# 範例工具用法 (可選測試)
print(get_weather("New York"))
print(get_weather("Paris"))

---

**2\. 定義代理程式 (`weather_agent`)**

現在，讓我們建立**代理程式**本身。ADK 中的 `Agent` 負責協調使用者、LLM 和可用工具之間的互動。

我們為其設定了幾個關鍵參數：

* `name`：此代理程式的唯一識別碼 (例如，「weather\_agent\_v1」)。  
* `model`：指定要使用的 LLM (例如，`MODEL_GEMINI_2_0_FLASH`)。我們將從一個特定的 Gemini 模型開始。  
* `description`：代理程式整體目的的簡潔摘要。當其他代理程式需要決定是否將任務委派給*此*代理程式時，這會變得至關重要。  
* `instruction`：關於如何行為、其角色、目標以及特別是*如何以及何時*利用其指派的 `tools` 的詳細指導。  
* `tools`：一個包含代理程式允許使用的實際 Python 工具函式的列表 (例如，`[get_weather]`)。

**最佳實踐：** 提供清晰且具體的 `instruction` 提示。指令越詳細，LLM 就越能理解其角色以及如何有效地使用其工具。如果需要，請明確說明錯誤處理。

**最佳實踐：** 選擇具描述性的 `name` 和 `description` 值。這些由 ADK 內部使用，對於像自動委派 (稍後介紹) 這樣的功能至關重要。

In [ ]:
# @title 定義天氣代理程式

weather_agent = Agent(
    name="weather_agent_v1",
    model=MODEL_GEMINI_2_0_FLASH,
    description="提供特定城市的天氣資訊。",
    instruction="您是一個樂於助人的天氣助理。"
                "當使用者詢問特定城市的天氣時，"
                "請使用 'get_weather' 工具尋找資訊。"
                "如果工具傳回錯誤，請禮貌地通知使用者。"
                "如果工具成功，請清楚地呈現天氣報告。"
                "您可以使用 preload memory 工具幫助使用者根據他們的偏好找到天氣較好的城市。",
    tools=[get_weather, adk.tools.preload_memory_tool.PreloadMemoryTool()], # 直接傳遞函式
)

print(f"代理程式 '{weather_agent.name}' 已使用模型 '{MODEL_GEMINI_2_0_FLASH}' 建立。")

---

**3\. 定義代理程式引擎**

代理程式引擎是一組服務，使開發人員能夠在生產環境中部署、管理和擴展 AI 代理程式。代理程式引擎處理基礎設施以在生產環境中擴展代理程式，因此您可以專注於建立應用程式。在 Vertex Express 模式下，我們可以免費使用某些代理程式引擎服務，主要是會話和記憶體服務，這些服務允許進行上下文管理。每個會話和記憶體都與一個代理程式引擎相關聯。

我們為我們的代理程式引擎設定了幾個關鍵參數：

* `displayName`：此代理程式引擎的唯一識別碼 (例如，「weather\_agent\_v1」)。  
* `description`：代理程式引擎整體目的的簡潔摘要。這可以幫助您記住它的作用。

In [ ]:
# @title 建立代理程式引擎
from google import genai
import json

# 使用 GenAI SDK 建立代理程式引擎
client = genai.Client(vertexai = True)._api_client
string_response = client.request(
        http_method='POST',
        path=f'reasoningEngines',
        request_dict={"displayName": "Express-Mode-Agent-Engine", "description": "測試代理程式引擎示範"},
    ).body
response = json.loads(string_response)
response

In [ ]:
APP_NAME="/".join(response['name'].split("/")[:6])
APP_NAME

In [ ]:
APP_ID=APP_NAME.split('/')[-1]
APP_ID

---

**4\. 設定 Runner 和會話服務**

若要管理對話並執行代理程式，我們還需要兩個元件：

* `SessionService`：負責管理不同使用者和會話的對話歷史和狀態。`VertexAiSessionService` 是一個將所有內容儲存在 Vertex 中的實作，允許持久性會話儲存。它會追蹤交換的訊息。我們將在步驟 4 中更深入地探討狀態持久性。  
* `Runner`：協調互動流程的引擎。它接收使用者輸入，將其路由到適當的代理程式，根據代理程式的邏輯管理對 LLM 和工具的呼叫，透過 `SessionService` 處理會話更新，並產生代表互動進度的事件。

In [ ]:
# @title 建立我們的初始會話

# 透過 ADK 建立 Vertex AI 會話
session_service = VertexAiSessionService(agent_engine_id=APP_ID)
memory_service = VertexAiMemoryBankService(agent_engine_id=APP_ID)

USER_ID = "INSERT_USER_ID" #@param {type:"string"}
session = await session_service.create_session(app_name=APP_ID, user_id=USER_ID)
SESSION_ID = session.id
session

In [ ]:
# @title 建立代理程式執行器

# 與 ADK 連接。ADK 也將使用 easygcp 金鑰來產生內容
print(f"會話已建立：應用程式='{APP_ID}'，使用者='{USER_ID}'，會話='{SESSION_ID}'")
# --- Runner ---
# 核心概念：Runner 協調代理程式執行迴圈。
runner = Runner(
    agent=weather_agent, # 我們要執行的代理程式
    app_name=APP_ID,   # 將執行與我們的應用程式關聯
    session_service=session_service, # 使用 vertex 會話服務
    memory_service=memory_service # 使用 vertex 記憶體服務
)
print(f"已為代理程式 '{runner.agent.name}' 建立 Runner。")

---

**5\. 與代理程式互動**

我們需要一種方法來向我們的代理程式傳送訊息並接收其回應。由於 LLM 呼叫和工具執行可能需要時間，ADK 的 `Runner` 是以非同步方式運作的。

我們將定義一個 `async` 輔助函式 (`call_agent_async`)，它會：

1. 接收一個使用者查詢字串。  
2. 將其打包成 ADK `Content` 格式。  
3. 呼叫 `runner.run_async`，提供使用者/會話上下文和新訊息。  
4. 迭代由 runner 產生的**事件**。事件代表代理程式執行中的步驟 (例如，請求工具呼叫、收到工具結果、中間的 LLM 思考、最終回應)。  
5. 使用 `event.is_final_response()` 識別並列印**最終回應**事件。

**為何使用 `async`？** 與 LLM 和潛在工具 (如外部 API) 的互動是 I/O 密集型操作。使用 `asyncio` 可以讓程式有效地處理這些操作，而不會阻塞執行。

In [ ]:
# @title 定義代理程式互動函式

from google.genai import types # 用於建立訊息內容/部分

async def call_agent_async(query: str, runner, user_id, session_id):
  """向代理程式傳送查詢並列印最終回應。"""
  print(f"\n>>> 使用者查詢：{query}")

  # 以 ADK 格式準備使用者訊息
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "代理程式未產生最終回應。" # 預設值

  # 核心概念：run_async 執行代理程式邏輯並產生事件。
  # 我們迭代事件以尋找最終答案。
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # 您可以取消註解以下這行以查看執行期間的*所有*事件
      print(f"  [事件] 作者：{event.author}，類型：{type(event).__name__}，最終：{event.is_final_response()}，內容：{event.content}")

      # 核心概念：is_final_response() 標記該輪次的結束訊息。
      if event.is_final_response():
          if event.content and event.content.parts:
             # 假設文字回應在第一部分
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # 處理潛在的錯誤/升級
             final_response_text = f"代理程式已升級：{event.error_message or '無特定訊息。'}"
          # 如果需要，在此處新增更多檢查 (例如，特定的錯誤代碼)
          break # 找到最終回應後停止處理事件

  print(f"<<< 代理程式回應：{final_response_text}")

---

**6\. 執行對話**

最後，讓我們透過向代理程式傳送幾個查詢來測試我們的設定。我們將 `async` 呼叫包裝在一個主 `async` 函式中，並使用 `await` 執行它。

觀察輸出：

* 查看使用者查詢。  
* 注意當代理程式使用工具時的 `--- 工具：呼叫 get_weather... ---` 日誌。  
* 觀察代理程式標記為「代理程式回應」的最終回應，包括它如何處理天氣資料不可用 (對於巴黎) 的情況。

In [ ]:
# @title 執行初始對話

# 我們需要一個 async 函式來等待我們的互動輔助函式
async def run_conversation():
    await call_agent_async("倫敦的天氣如何？",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

    await call_agent_async("巴黎呢？",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID) # 預期工具的錯誤訊息

    await call_agent_async("告訴我紐約的天氣",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)
    await call_agent_async("我更喜歡紐約的天氣，聽起來比倫敦的天氣好",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)
    await call_agent_async("我之前問過你哪些城市？",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

# 在非同步上下文中使用 await 執行對話 (如 Colab/Jupyter)
await run_conversation()

---

恭喜！您已成功建構並與您的第一個 ADK 代理程式互動，並免費使用了 Vertex 會話服務！

---

**8\. 測試代理程式記憶體**

讓我們看看代理程式是否會記住我們在先前會話中的偏好。
- 在這裡，我們可以建立一個新的會話，然後詢問代理程式我們在先前會話中談論過的事情。在此範例中，我們在先前的對話中談到偏好紐約的天氣後，詢問我們的天氣偏好。
- 代理程式應利用 vertex 記憶體服務來檢索有關使用者的相關詳細資訊，然後在其回應中利用這些資訊。

In [ ]:
# @title 根據先前的會話建立記憶體

# 我們可以根據先前的會話 ID 產生記憶體
session = await session_service.get_session(app_name=APP_ID, session_id=SESSION_ID, user_id = USER_ID)
await memory_service.add_session_to_memory(session=session)

In [ ]:
# @title 測試代理程式記憶體

# 建立一個新的會話，讓我們看看它是否會根據我們的使用者 ID 記住我們的偏好
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID)
SESSION_ID = session.id

print(f"新會話已建立：應用程式='{APP_NAME}'，使用者='{USER_ID}'，會話='{SESSION_ID}'")

await call_agent_async("根據我的偏好，哪個城市的天氣最適合我？",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)